In [1]:
import pandas as pd 
import os
import sys

sys.path.insert(0, '../')
from scripts.tracking import TeamTracking 

In [3]:
data_directory = '../data/tracking'
file = 'g2312201_SecondSpectrum_tracking-produced.jsonl'
match_tracking_path = os.path.join(data_directory, file)

# < 20s
df_unstructured_tracking = pd.read_json(match_tracking_path, lines = True)

In [4]:
df_unstructured_tracking.head()

,period,frameIdx,gameClock,wallClock,homePlayers,awayPlayers,ball,live,lastTouch
0,1,0,0.00,1674387294150,[{'playerId': '02780bab-8db4-4310-ad80-308ae2f...,[{'playerId': '25357429-779e-43bb-aa85-8c2ef9c...,"{'xyz': [-0.58, 0.08, 0.34], 'speed': 8.97}",False,home
1,1,1,0.04,1674387294190,[{'playerId': '02780bab-8db4-4310-ad80-308ae2f...,[{'playerId': '25357429-779e-43bb-aa85-8c2ef9c...,"{'xyz': [-0.89, 0.09, 0.35000000000000003], 's...",True,away
2,1,2,0.08,1674387294230,[{'playerId': '02780bab-8db4-4310-ad80-308ae2f...,[{'playerId': '25357429-779e-43bb-aa85-8c2ef9c...,"{'xyz': [-1.2, 0.09, 0.36], 'speed': 9.11}",True,away
3,1,3,0.12,1674387294270,[{'playerId': '02780bab-8db4-4310-ad80-308ae2f...,[{'playerId': '25357429-779e-43bb-aa85-8c2ef9c...,"{'xyz': [-1.52, 0.1, 0.37], 'speed': 9.15}",True,away
4,1,4,0.16,1674387294310,[{'playerId': '02780bab-8db4-4310-ad80-308ae2f...,[{'playerId': '25357429-779e-43bb-aa85-8c2ef9c...,"{'xyz': [-1.85, 0.11, 0.37], 'speed': 9.15}",True,away


In [5]:
# Traitement de 1,5M de lignes en < 2s
HomeTracking = TeamTracking(df_unstructured_tracking=df_unstructured_tracking)
HomeTracking.df_tracking

,period,frameIdx,gameClock,wallClock,live,lastTouch,optaId,speed,xyz
0,1,0,0.00,1674387294150,False,home,246319,0.00,"[16.93, 16.79, 0.0]"
0,1,0,0.00,1674387294150,False,home,192194,0.00,"[13.61, 0.11, 0.0]"
0,1,0,0.00,1674387294150,False,home,174349,0.00,"[3.04, 17.82, 0.0]"
0,1,0,0.00,1674387294150,False,home,96385,0.00,"[6.94, -8.83, 0.0]"
0,1,0,0.00,1674387294150,False,home,459038,0.00,"[5.21, 9.58, 0.0]"
...,...,...,...,...,...,...,...,...,...
144074,2,144074,2882.04,1674394107470,False,away,165677,0.77,"[-9.2, 14.65, 0.0]"
144074,2,144074,2882.04,1674394107470,False,away,175128,0.18,"[2.95, 2.06, 0.0]"
144074,2,144074,2882.04,1674394107470,False,away,447080,1.08,"[-11.49, 7.74, 0.0]"
144074,2,144074,2882.04,1674394107470,False,away,96535,2.00,"[-8.57, 21.25, 0.0]"
